In [37]:
import pandas as pd
import numpy as np

In [105]:
df = pd.read_csv("Options_Nov29.csv")
df.head()

,No.,Trade date,Type,Name,Ticker,Buy/Sell,Call/Put,Instrument,Currency,Spot price,...,Expiry day,Stock Qty,Ntl size (mn),Adj. Ntl - USD,Broker,% premium,Premium (USD),Status,PNL,DTE
0,65,2023-01-03,OTC - Stock,NaN,UMG NA,Sell,Call,Sell Call,EUR,22.675,...,2023-02-27,122353,2.848378,3.418543,MS,0.003999,11391.06000,Expired,0.0,55
1,66,2023-01-03,OTC - Stock,NaN,NTES US,Sell,Call,Sell Call,USD,89.900,...,2023-02-03,13000,1.003495,1.204194,GS,0.006795,6819.18000,Expired,0.0,31
2,67,2023-01-04,OTC - Stock,NaN,002706 CH,Sell,Call,Sell Call,CNY,13.710,...,2023-03-06,469000,6.913060,1.202007,MS,0.015996,16021.04000,Expired,0.0,61
3,68,2023-01-04,OTC - Stock,NaN,ADYEN NA,Sell,Put,Sell Put,EUR,1466.800,...,2023-02-17,757,0.999338,0.849437,GS,0.016500,16489.07241,Expired,0.0,44
4,69,2023-01-04,OTC - Stock,NaN,ADYEN NA,Sell,Put,Sell Put,EUR,1466.800,...,2023-02-17,757,0.999338,0.799470,GS,0.010000,9993.37653,Expired,0.0,44


In [106]:
# month by month
df['Trade date'] = pd.to_datetime(df['Trade date'])

grouped = df.groupby(pd.Grouper(key="Trade date",freq="M"))

monthly_dfs = [group for _, group in grouped]

def counts(df):
    num_trades = len(df)
    total_premium = sum(df['Premium (USD)'])
    total_pnl = sum(df['PNL'])
    return ([num_trades, total_premium, total_pnl])

d = {}

for i in range(0, 11):
    d[i] = counts(df = monthly_dfs[i])

d = pd.DataFrame.from_dict(d, orient='index', columns=['Numebr of Trade', 'Premium', 'PNL'])
d.index = [1,2,3,4,5,6,7,8,9,10,11]
d

,Numebr of Trade,Premium,PNL
1,58,8.524072e+05,-1.830367e+05
2,67,9.136963e+05,-2.190744e+06
3,81,1.110729e+06,-1.065598e+06
4,48,2.671659e+05,-1.762515e+05
5,34,3.303308e+05,-1.621859e+05
6,47,2.612192e+05,-2.170250e+05
7,69,9.852339e+05,-7.272089e+05
8,54,9.200307e+05,-2.146667e+06
9,54,8.387500e+05,-8.599417e+05
10,59,6.920818e+05,-1.764726e+05


In [119]:
# name by name

df = pd.read_csv("Options_Nov29.csv")

grouped = df.groupby('Ticker')
ticker_dfs = [group for _, group in grouped]

# find which strike group this contract belongs to
def strikes_group(n):
    ranges = [[0,0.8],[0.8,0.85],[0.85,0.9],[0.9,0.95],[0.95,1],[1,1.05], [1.05,1.1], [1.1,1.15], [1.15,1.2], [1.2,2]]
    name = ['<80%','80% - 84%','85% - 89%','90% - 94%','95% - 99%','100% - 104%', '105% - 109%', '110% - 114%','115% - 119%' , '>120%']
    group = None
    for i, range_ in enumerate(ranges):
        if range_[0] <= n < range_[1]:
            group = i
            break
    return name[group]

def all_hit_rate(df):
    win_1 = 0
    total_1 = 0
    win_2 = 0
    total_2 = 0
    win_3 = 0
    total_3 = 0
    for i in range(len(df)):
        if df.iloc[i]['DTE'] <= 16:
            if df.iloc[i]['status'] == 'Expired' or df.iloc[i]['status'] == 'Unwound':
                win_1 += 1
                total_1 += 1
            elif df.iloc[i]['status'] == 'Exercised':
                total_1 += 1
        elif df.iloc[i]['DTE'] > 16 and df.iloc[i]['DTE'] <= 33:
            if df.iloc[i]['status'] == 'Expired' or df.iloc[i]['status'] == 'Unwound':
                win_2 += 1
                total_2 += 1
            elif df.iloc[i]['status'] == 'Exercised':
                total_2 += 1
        else:
            if df.iloc[i]['status'] == 'Expired' or df.iloc[i]['status'] == 'Unwound':
                win_3 += 1
                total_3 += 1
            elif df.iloc[i]['status'] == 'Exercised':
                total_3 += 1
    less_2_weeks_hit_rate = win_1 / total_1
    less_1_month_hit_rate = win_2 / total_2
    more_1_month_hit_rate = win_3 / total_3

    return({'less_2_weeks_hit_rate': less_2_weeks_hit_rate, 'less_1_month_hit_rate': less_1_month_hit_rate, 'more_1_month_hit_rate': more_1_month_hit_rate})
    
def hit_rate(df):
    n_exercised = df['Status'].value_counts().get('Exercised',0)
    n_expired = df['Status'].value_counts().get('Expired',0)
    n_unwounded = df['Status'].value_counts().get('Unwound',0)
    hit_rate = (n_expired + n_unwounded) / (n_expired + n_unwounded + n_exercised)
    return hit_rate

result = {}
for i in range(len(ticker_dfs)):
    df = ticker_dfs[i]
    ticker = df.iloc[0]['Ticker']
    overall_hit_rate = hit_rate(df=df)

    # notional traded
    notl = df['Adj. Ntl - USD'].sum()

    # premiums received
    prem = df['Premium (USD)'].sum()

    # pnl
    pnl = df['PNL'].sum()

    # directions and hit rates
    # check if this ticker has done this direction
    sellcall_hit_rate = 'NA'
    sellput_hit_rate = 'NA'
    buycall_hit_rate = 'NA'
    buyput_hit_rate = 'NA'


    if 'Sell Call' in df['Instrument']:
        sellcall = df[df['Instrument'] == 'Sell Call']
        sellcall_hit_rate = hit_rate(df=sellcall)
    elif 'Sell Put' in df['Instrument']:
        sellput = df[['Instrument'] == 'Sell Put']
        sellput_hit_rate = hit_rate(df=sellput)
    elif 'Buy Call' in df['Instrument']:
        buycall = df[['Instrument'] == 'Buy Call']
        buycall_hit_rate = hit_rate(df=buycall)
    elif 'Buy Put' in df['Instrument']:
        buyput = df[['Instrument'] == 'Buy Put']
        buyput_hit_rate = hit_rate(df=buyput)
        

    result[ticker] = [notl, prem, pnl, overall_hit_rate, sellcall_hit_rate, sellput_hit_rate, buycall_hit_rate, buyput_hit_rate]

In [120]:
col = ['notl', 'prem', 'pnl', 'overall_hit_rate', 'sellcall_hit_rate', 'sellput_hit_rate', 'buycall_hit_rate', 'buyput_hit_rate']
result = pd.DataFrame.from_dict(data = result, orient='index')
result.columns = col
result

,notl,prem,pnl,overall_hit_rate,sellcall_hit_rate,sellput_hit_rate,buycall_hit_rate,buyput_hit_rate
002050 CH,19.357607,2.410724e+05,-9.433467e+03,0.944444,NA,NA,NA,NA
002706 CH,1.202007,1.602104e+04,0.000000e+00,1.000000,NA,NA,NA,NA
300012 CH,5.399773,5.003853e+04,-2.070692e+05,0.166667,NA,NA,NA,NA
300059 CH,16.782306,1.496573e+05,0.000000e+00,1.000000,NA,NA,NA,NA
300285 CH,4.397852,4.003255e+04,0.000000e+00,1.000000,NA,NA,NA,NA
300470 CH,4.893836,4.754059e+04,0.000000e+00,1.000000,NA,NA,NA,NA
301029 CH,4.669915,6.268377e+04,0.000000e+00,1.000000,NA,NA,NA,NA
688036 CH,10.343752,1.245017e+05,-9.008390e+05,0.333333,NA,NA,NA,NA
688166 CH,2.653773,9.409705e+04,-7.533764e+04,0.500000,NA,NA,NA,NA
688392 CH,20.932735,3.402988e+05,-1.458079e+05,0.687500,NA,NA,NA,NA


In [135]:
df = ticker_dfs[0]
ticker = df.iloc[0]['Ticker']
overall_hit_rate = hit_rate(df=df)

# notional traded
notl = df['Adj. Ntl - USD'].sum()

# premiums received
prem = df['Premium (USD)'].sum()

# pnl
pnl = df['PNL'].sum()

# directions and hit rates
# check if this ticker has done this direction
sellcall_hit_rate = 0
sellput_hit_rate = 'NA'
buycall_hit_rate = 'NA'
buyput_hit_rate = 'NA'


if 'Sell Call' in df['Instrument']:
    sellcall = df[df['Instrument'] == 'Sell Call']
    sellcall_hit_rate = hit_rate(df=sellcall)
elif 'Sell Put' in df['Instrument']:
    sellput = df[['Instrument'] == 'Sell Put']
    sellput_hit_rate = hit_rate(df=sellput)
elif 'Buy Call' in df['Instrument']:
    buycall = df[['Instrument'] == 'Buy Call']
    buycall_hit_rate = hit_rate(df=buycall)
elif 'Buy Put' in df['Instrument']:
    buyput = df[['Instrument'] == 'Buy Put']
    buyput_hit_rate = hit_rate(df=buyput)

In [136]:
sellcall_hit_rate

0

In [131]:
xxx

0.6666666666666666

In [71]:
not_hit = ['Expired','Unwound']
hit_rate = df['Status'].value_counts()['Expired'] 

In [72]:
hit_rate

17

In [75]:
hit_rate = df['Status'].value_counts()['Exercised']
hit_rate

1